# Listings

In [1]:
import pandas as pd

df_2020_03 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2020-03-16/data/listings.csv.gz', compression='gzip')
df_2019_18 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2019-10-18/data/listings.csv.gz', compression='gzip')
df_2018_10 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2018-10-11/data/listings.csv.gz', compression='gzip')
df_2017_10 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2017-10-06/data/listings.csv.gz', compression='gzip')
df_2016_09 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2016-09-07/data/listings.csv.gz', compression='gzip')
df_2015_10 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2015-10-03/data/listings.csv.gz', compression='gzip')

df_original_listings = pd.concat([df_2020_03, df_2019_18, df_2018_10, df_2017_10, df_2016_09, df_2015_10]).drop_duplicates('id')
print('Len total =', len(df_original_listings))

Len total = 12480


In [2]:
df_listings = df_original_listings
df_listings = df_listings.replace('nan', '', regex=True)
df_listings['summary'] = df_listings['summary'].astype(str)
df_listings['space'] = df_listings['space'].astype(str)
df_listings['description'] = df_listings['description'].astype(str)
df_listings['neighborhood_overview'] = df_listings['neighborhood_overview'].astype(str)
df_listings['host_neighbourhood'] = df_listings['host_neighbourhood'].astype(str)
df_listings['neighbourhood_cleansed'] = df_listings['neighbourhood_cleansed'].astype(str)
df_listings['notes'] = df_listings['notes'].astype(str)

In [3]:
from collections import Counter
from itertools import chain
import itertools
import string
import re

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english'))

def get_neighborhood_adjectives(name):
  neigh = df_listings.loc[df_listings['neighbourhood_cleansed'] == name]

  overviews = [n.split() for n in neigh['neighborhood_overview']]
  adjectives = []
  for overview in overviews:
    useless_words = ['many', 'great', 'nearby', 'short', 'major', 'north', 'south', 'east', 'west', 'easy', 'true', 'local', 'public', 'mi', 'several', 'such', 'main', 'other']
    text = [word for word in overview if word.lower() not in useless_words]
    text = re.sub('[^\w\s]+', '', ' '.join(text))
    tagged = nltk.FreqDist(nltk.pos_tag(text.split()))
    jj = [wt[0] for (wt, _) in tagged.most_common() if wt[1] == 'JJ']
    adjectives.append(jj)

  flatAdjectives = list(itertools.chain.from_iterable(adjectives))

  counter = Counter(flatAdjectives)
  return (len(neigh), counter)

# north_end = get_neighborhood_adjectives('North End')
# print('In north end:', north_end[0])
# print('North end is:', north_end[1])

[nltk_data] Downloading package stopwords to /Users/marc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/marc/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
# for neighborhood in df_listings['neighbourhood_cleansed'].unique():
#     ads = get_neighborhood_adjectives(neighborhood)
#     print(neighborhood, ':', ads[0])
#     print(neighborhood, 'is:', ads[1].most_common(5))

# Reviews

In [5]:
import pandas as pd

reviews_2020_03 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2020-03-16/data/reviews.csv.gz', compression='gzip')
reviews_2019_18 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2019-10-18/data/reviews.csv.gz', compression='gzip')
reviews_2018_10 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2018-10-11/data/reviews.csv.gz', compression='gzip')
reviews_2017_10 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2017-10-06/data/reviews.csv.gz', compression='gzip')
reviews_2016_09 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2016-09-07/data/reviews.csv.gz', compression='gzip')
reviews_2015_10 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2015-10-03/data/reviews.csv.gz', compression='gzip')

df_reviews = pd.concat([reviews_2020_03, reviews_2019_18, reviews_2018_10, reviews_2017_10, reviews_2016_09, reviews_2015_10])
df_reviews['comments'] = df_reviews['comments'].astype(str)
print('Len total =', len(df_reviews))

Len total = 794377


In [6]:
df_reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3781,37776825,2015-07-10,36059247,Greg,The apartment was as advertised and Frank was ...
1,3781,41842494,2015-08-09,10459388,Tai,It was a pleasure to stay at Frank's place. Th...
2,3781,45282151,2015-09-01,12264652,Damien,The apartment description is entirely faithful...
3,3781,49022647,2015-09-30,41426327,Mike,Thoroughly enjoyed my time at Frank's home. Ha...
4,3781,52503327,2015-10-30,15151513,Ivan,Great value for the money! This location has e...
...,...,...,...,...,...,...
43118,6350149,43556545,2015-08-20,8581396,Atanas,"Bernie was a great host, everything went very ..."
43119,6350149,43731210,2015-08-21,41736991,Edd,First time using AirBNB. Needed a last minute ...
43120,6350149,44353669,2015-08-25,40994591,Michael,Overall: Disappointed with expectation from ph...
43121,6350149,44708580,2015-08-28,41152147,François,We did not see Bernie. One of his room mate we...


# Vibe Vectors

In [11]:
vibedict = []

for neighborhood in df_listings['neighbourhood_cleansed'].unique():
  neighborhood_adjectives = get_neighborhood_adjectives(neighborhood)[1]
  adjectives = list(neighborhood_adjectives)
  frequencies = [neighborhood_adjectives[adjective] for adjective in adjectives]

  # for each listing within this neighborhood
  for listing_id in df_listings.loc[df_listings['neighbourhood_cleansed'] == neighborhood]['id']:
    comments_list = df_reviews.loc[df_reviews['listing_id'] == listing_id]['comments'].tolist()
    if len(comments_list) == 0:
      continue

    # Neighborhood accuracy is essentially the dot product between the neighborhood description
    # and the reviews left for the place. This indicates the accuracy of the neighborhood
    # description and the vibe that people who stayed felt.

    # Neighborhood accuracy is the sum of the number of adjectives that appear 
    # in both the neighborhood description and in the reviews for a listing,
    # weighted by the frequency by which those adjectives occur in all 
    # neighborhood descriptions within that neighborhood. 
    neighborhood_accuracy = 0

    listing_comments = re.sub('[^\w\s]+', '', ' '.join(comments_list)).split()
    comment_freq = Counter(listing_comments)
    comment_words = list(comment_freq.keys())

    for i in range(len(adjectives)):
      if (adjectives[i] in comment_words):
        neighborhood_accuracy += frequencies[i] * comment_freq[adjectives[i]]
        # neighborhood_accuracy += 1

    neighborhood_accuracy /= len(comments_list)
    vibedict.append({'lat': df_listings.loc[df_listings['id'] == listing_id]['latitude'].values[0],
                     'lon': df_listings.loc[df_listings['id'] == listing_id]['longitude'].values[0],
                     'rating': df_listings.loc[df_listings['id'] == listing_id]['review_scores_rating'].values[0],
                     'listing_id': listing_id, 
                     'neighborhood_accuracy': neighborhood_accuracy, 
                     'num_reviews': len(comments_list)})

vibetable = pd.DataFrame(columns=['listing_id', 'neighborhood_accuracy', 'num_reviews'])
vibetable = vibetable.append(vibedict, ignore_index=True)

In [12]:
vibetable.to_csv("dist/data/vibetable.csv", index=False)
vibetable.to_json("dist/data/vibetable.json", orient='index')

In [13]:
vibetable

,listing_id,neighborhood_accuracy,num_reviews,lat,lon,rating
0,3781,64.098361,61,42.364130,-71.029910,99.0
1,1042420,52.250597,419,42.383620,-71.020230,94.0
2,3965313,27.312741,259,42.386790,-71.003020,88.0
3,4315428,56.000000,3,42.390670,-71.003860,80.0
4,4315439,44.692308,13,42.389210,-71.002680,88.0
...,...,...,...,...,...,...
9823,4645486,0.547619,42,42.350167,-71.057112,95.0
9824,9229058,1.833333,6,42.350614,-71.056625,100.0
9825,8733896,1.162162,37,42.351448,-71.058670,100.0
9826,215039,0.818182,110,42.351641,-71.057128,99.0


In [10]:
list(df_listings)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [43]:
df_listings['reviews_per_month']

0       0.28
1       0.79
2       0.89
3       0.35
4       0.24
        ... 
2553    2.81
2554    3.80
2555    2.11
2556    2.00
2557    0.64
Name: reviews_per_month, Length: 12480, dtype: float64

# Stays

In [14]:
stays_2020_03 = pd.read_csv('http://data.insideairbnb.com/united-states/ma/boston/2020-03-16/data/calendar.csv.gz', compression='gzip')

In [15]:
stays_2020_03

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,3781,2020-03-16,f,$125.00,$125.00,28,1125
1,2187766,2020-03-16,t,$519.00,$519.00,29,1125
2,2187766,2020-03-17,t,$419.00,$419.00,29,1125
3,2187766,2020-03-18,t,$419.00,$419.00,29,1125
4,2187766,2020-03-19,t,$349.00,$349.00,29,1125
...,...,...,...,...,...,...,...
1386630,41615014,2021-03-11,t,$133.00,$133.00,31,1125
1386631,41615014,2021-03-12,t,$133.00,$133.00,31,1125
1386632,41615014,2021-03-13,t,$133.00,$133.00,31,1125
1386633,41615014,2021-03-14,t,$133.00,$133.00,31,1125


In [17]:
list(df_reviews)

['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']

In [20]:
df_reviews


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3781,37776825,2015-07-10,36059247,Greg,The apartment was as advertised and Frank was ...
1,3781,41842494,2015-08-09,10459388,Tai,It was a pleasure to stay at Frank's place. Th...
2,3781,45282151,2015-09-01,12264652,Damien,The apartment description is entirely faithful...
3,3781,49022647,2015-09-30,41426327,Mike,Thoroughly enjoyed my time at Frank's home. Ha...
4,3781,52503327,2015-10-30,15151513,Ivan,Great value for the money! This location has e...
...,...,...,...,...,...,...
43118,6350149,43556545,2015-08-20,8581396,Atanas,"Bernie was a great host, everything went very ..."
43119,6350149,43731210,2015-08-21,41736991,Edd,First time using AirBNB. Needed a last minute ...
43120,6350149,44353669,2015-08-25,40994591,Michael,Overall: Disappointed with expectation from ph...
43121,6350149,44708580,2015-08-28,41152147,François,We did not see Bernie. One of his room mate we...
